## <center> Bank Of America Case </center>

### Import Data and Storage in Hash Table

In [80]:
import numpy as np
import pandas as pd
import os
import re

In [81]:
def fix_first_row(dic_data, key):
    arr = np.array([])
    for idx, value in enumerate(dic_data[key].iloc[0:1,:].values.reshape(-1,1).flatten()):
        if idx != 1:
            lista = value.split(" ")
            if len(arr) == 0:
                arr = np.hstack((arr, np.array(lista)))
                arr = arr.reshape(-1,1)
                continue
            arr = np.hstack((arr, np.array(lista).reshape(-1,1)))
            continue
        lista = value.split("Zelle")
        final_string = lista[len(lista) - 1][:0] + 'Zelle' + lista[len(lista) - 1][0:]
        lista[len(lista) - 1] = final_string
        arr = np.hstack((arr, np.array(lista).reshape(-1,1)))
    return pd.DataFrame(arr, columns = ["DATE", "DESCRIPTION", "AMOUNT"])

In [82]:
path = os.getcwd()
folder = "output_bank_of_america_documentai"
folder_file = "file_6"
files = os.listdir(os.path.join(folder, folder_file))

In [83]:
dic_data = {}
for file in files:
    data = pd.read_csv(os.path.join(path, folder, folder_file, file))
    p = "(pg\d\w+)"
    regex_p= re.compile(p)
    dic_key = regex_p.search(file).group()
    dic_data[dic_key] = data

In [84]:
for key in dic_data.keys():
    print(key)

pg1_tb0
pg3_tb0
pg4_tb0
pg4_tb1
pg4_tb2


In [87]:
dic_data["pg4_tb0"]

,Unnamed: 0,Total for this period,Total year-to-date
0,Total Overdraft fees,$0.00,$70.00
1,Total NSF: Returned Item fees,$0.00,$0.00


### Separate used table and Storage in Another Dic

In [7]:
tables = {}
for key in dic_data.keys():
    if len(dic_data[key].columns) == 3 and "Description" in dic_data[key].columns:
        tables[key] = dic_data[key] 
    # elif len(dic_data[key].columns) == 3 and "Amount" in dic_data[key].columns:
    #     tables[key] = dic_data[key]


In [8]:
for key in tables.keys():
    print(key)

eStmt_2021-01-29 (1)_pg3_tb0
eStmt_2021-01-29 (1)_pg3_tb1
eStmt_2021-01-29 (1)_pg4_tb0


In [9]:
dic_data["eStmt_2021-01-29 (1)_pg4_tb0"]

,Date,Description,Amount
0,01/12/21 01/13/21,AMERICAN EXPRESS DES:ACH PMT ID:M4010 INDN:Riv...,"-1,132.32 -200.00"
1,01/15/21,Zelle Transfer Conf# 607f94355; Jhonattan Chica,-800.00
2,01/19/21,AMERICAN EXPRESS DES:ACH PMT ID:M1438 INDN:Riv...,"-9,000.00"
3,01/19/21,"AUDI FINCL, TEL. DES:WEB DEBIT ID: PWBS8116261...",-865.93
4,01/19/21,DES:Auto Pay ID:5000726658 INDN:River Luna CO ...,-583.88
5,01/20/21,VERIZON WIRELESS DES:PAYMENTS ID:0325517165000...,-555.38
6,01/25/21,Zelle Transfer Conf# 4dde984b8; Jhonattan Chica,-500.00
7,01/26/21,Zelle Transfer Conf# 739b24f62; Carlos Zapata,-50.00
8,01/28/21,Zelle Transfer Conf# 55e1711e2; Diego Reloj,"-1,000.00"
9,01/28/21,GM Financial DES:GMF Pymt ID:00112043538669 IN...,-718.42


### Preprocessing tables 

In [10]:
tables_2 = {}
for key in tables.keys():
    if "credits" in tables[key].Description.values[len(tables[key]) - 1]:
        
        table = tables[key].astype(str)
        table.columns = [x.upper() for x in table.columns]
        
        table = table.drop(table.index[len(table)-1])
        
        table.AMOUNT = table.AMOUNT.apply(lambda x: x.replace("$", ""))
        table.AMOUNT = table.AMOUNT.apply(lambda x: x.replace(",", ""))
        table["AMOUNT"] = table["AMOUNT"].astype(float)
        table["type"] = "Income"
        
        tables_2[key] = table

    elif "debits" in tables[key].Description.values[len(tables[key]) - 1]:
        
        table = tables[key]
        table = table.dropna()
        table = table.astype(str)
        
        firsts_row = fix_first_row(tables, key)
        table = table.iloc[1:, :] #delete first row
        table.columns = ["DATE", "DESCRIPTION", "AMOUNT"]
        table = pd.concat([firsts_row,table], axis=0).reset_index(drop=True)
        
        while "total" in table.DATE.values[len(table) - 1] or "Total" in table.DATE.values[len(table) - 1]:
            table = table.drop(table.index[len(table)-1])
        
        
        
        table.AMOUNT = table.AMOUNT.apply(lambda x: x.replace("$", ""))
        table.AMOUNT = table.AMOUNT.apply(lambda x: x.replace(",", ""))
        table["AMOUNT"] = table["AMOUNT"].astype(float)
        table["type"] = "Withdrawals"

        tables_2[key] = table
        
    else:
        table = tables[key].astype(str)
        table.columns = [x.upper() for x in table.columns]
        table.AMOUNT = table.AMOUNT.apply(lambda x: x.replace("$", ""))
        table.AMOUNT = table.AMOUNT.apply(lambda x: x.replace(",", ""))
        table["AMOUNT"] = table["AMOUNT"].astype(float)
        if table["AMOUNT"].values[0] < 0:
            table["type"] = "Withdrawals"
        elif table["AMOUNT"].values[0] > 0:
            table["type"] = "Income"
        tables_2[key] = table

In [11]:
for key in tables_2.keys():
    print(key)

eStmt_2021-01-29 (1)_pg3_tb0
eStmt_2021-01-29 (1)_pg3_tb1
eStmt_2021-01-29 (1)_pg4_tb0


In [12]:
tables_2["eStmt_2021-01-29 (1)_pg4_tb0"]

,DATE,DESCRIPTION,AMOUNT,type
0,01/12/21,AMERICAN EXPRESS DES:ACH PMT ID:M4010 INDN:Riv...,-1132.32,Withdrawals
1,01/13/21,Zelle Transfer Conf# 8ea678a04; Samuel Rivera,-200.00,Withdrawals
2,01/15/21,Zelle Transfer Conf# 607f94355; Jhonattan Chica,-800.00,Withdrawals
3,01/19/21,AMERICAN EXPRESS DES:ACH PMT ID:M1438 INDN:Riv...,-9000.00,Withdrawals
4,01/19/21,"AUDI FINCL, TEL. DES:WEB DEBIT ID: PWBS8116261...",-865.93,Withdrawals
5,01/19/21,DES:Auto Pay ID:5000726658 INDN:River Luna CO ...,-583.88,Withdrawals
6,01/20/21,VERIZON WIRELESS DES:PAYMENTS ID:0325517165000...,-555.38,Withdrawals
7,01/25/21,Zelle Transfer Conf# 4dde984b8; Jhonattan Chica,-500.00,Withdrawals
8,01/26/21,Zelle Transfer Conf# 739b24f62; Carlos Zapata,-50.00,Withdrawals
9,01/28/21,Zelle Transfer Conf# 55e1711e2; Diego Reloj,-1000.00,Withdrawals


### Final Step and Compile all Information in 1 DataFrame

In [13]:
df_bank_of_america_final = pd.DataFrame()
for key in tables_2.keys():
    df_bank_of_america_final = pd.concat([df_bank_of_america_final, tables_2[key]], axis=0)
    df_bank_of_america_final = df_bank_of_america_final.sort_values(["type", "DATE"])

### Check Information

In [14]:
df_bank_of_america_final

,DATE,DESCRIPTION,AMOUNT,type
0,01/05/21,Zelle Transfer Conf# XXXXXXXXX; VALERIA LUNA G...,450.00,Income
1,01/07/21,Zelle Transfer Conf# T09LBJ3SM; CAMILO GOMEZ,180.00,Income
2,01/08/21,Zelle Transfer Conf# XXXXXXXXX; Jose R Puig,584.00,Income
3,01/12/21,BKOFAMERICA ATM 01/12 #000002974 DEPOSIT IVES ...,5000.00,Income
4,01/25/21,BKOFAMERICA ATM 01/25 #000007445 DEPOSIT WASHI...,1000.00,Income
5,01/28/21,BKOFAMERICA ATM 01/28 #000002120 DEPOSIT WILLI...,3180.00,Income
6,01/29/21,BKOFAMERICA ATM 01/28 #000004114 DEPOSIT IVES ...,1900.00,Income
0,01/04/21,Zelle Transfer Conf# ed9fff0a1; Mayerlin Ramirez,-1200.00,Withdrawals
1,01/04/21,Zelle Transfer Conf# 588ed71ee; Faelo,-300.00,Withdrawals
2,01/04/21,Zelle Transfer Conf# c109a56a8; Diana Baldor,-134.00,Withdrawals


In [15]:
df_bank_of_america_final[df_bank_of_america_final.type == "Income"].AMOUNT.sum()

12294.0

In [16]:
df_bank_of_america_final[df_bank_of_america_final.type == "Withdrawals"].AMOUNT.sum()

-26498.879999999997

## Delete Names

In [17]:
names = ["VALERIA LUNA GONZALEZ",
"CAMILO GOMEZ",
"Jose R Puig",
"Mayerlin Ramirez",
"Faelo",
"Diana Baldor",
"Juan Rivera",
"Erminia Luna",
"JUAN RIVERA",
"JuanRivera",
"LUNA,VALERIA",
"LUNA, VALERIA",
"Jhonattan Chica",
"Vanesa Garcia", 
"Samuel Rivera", 
"Jhonattan Chica",
"VALERIA LUNA GONZALEZ", 
"River Luna",
"Jhonattan Chica",
"Carlos Zapata", 
"Diego Reloj"]

In [18]:
for name in names:
    df_bank_of_america_final.DESCRIPTION = df_bank_of_america_final.DESCRIPTION.apply(lambda x: x.replace(name, "#####"))

## Save Output

In [19]:
df_bank_of_america_final["bank"] = "Bank Of America"
folder_output = "output_script"
file = "bank_of_america_output.csv"

In [20]:
df_bank_of_america_final.to_csv(os.path.join(path,folder_output, file), index=False, encoding="utf8")